# Step 2: Run Inference on Full Trajectories

That was easy! Now let's try to run inference across a whole trajectory and visualize the results!

In [1]:
import os
from pathlib import Path
import sys

# !IMPORTANT!, else Hydra isnt working !
current_path = os.getcwd()
sys.path.insert(0, Path(current_path).absolute().parents[0].as_posix())

import cv2
import tensorflow_datasets as tfds
import tensorflow as tf
import tqdm
import mediapy
import numpy as np
import torch
from uha import make_pytorch_oxe_iterable_dataset, get_octo_dataset_tensorflow, get_single_dataset_tensorflow, get_sequential_dataset_tensorflow
from torchvision.transforms.functional import convert_image_dtype
import omegaconf
from omegaconf import OmegaConf

/home/marcelr/miniforge3/envs/uha_test_policy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-08 20:10:10.257729: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 20:10:10.257816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 20:10:10.259238: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-08 20:10:10.267650: I tensorflow/core/platform/cpu_feature_guard.cc:182] This

## Load Data
Next, we will load a trajectory from the Bridge dataset for testing the model. We will use the publicly available copy in the Open X-Embodiment dataset bucket.

In [2]:
cfg = omegaconf.OmegaConf.load("../uha/data/conf/uha_default_load_config.yaml")
cfg_transforms = omegaconf.OmegaConf.load("../uha/data/conf/transforms/oxe_no_remapping.yaml")

In [4]:
# dataset = get_octo_dataset_tensorflow(cfg, train=True)
dataset = get_single_dataset_tensorflow(cfg, train=True).repeat().unbatch()
# dataset = get_sequential_dataset_tensorflow(cfg, train=True)
is_single_dataset = True
batch_size = 128
# visual_dataset = (dataset.flatten().batch(batch_size))
# create Pytorch Train Dataset
dataloader = make_pytorch_oxe_iterable_dataset(dataset, train=True, batch_size=batch_size, transform_dict=cfg_transforms, num_workers=0, pin_memory=True, is_single_dataset=is_single_dataset, main_process=False)
it = iter(dataloader)
batch = next(it)
# print(batch)
# if batch["observation"]["image_primary"].shape[-1] == 3:
#   batch["observation"]["image_primary"] = torch.moveaxis(batch["observation"]["image_primary"], -1, -3) # move rgb to get [..., rgb, width, height]
# if batch["observation"]["image_primary"].dtype == torch.uint8:
#   batch["observation"]["image_primary"] = convert_image_dtype(batch["observation"]["image_primary"], dtype=torch.float32)
images, image_wrist = [], []
for i in range(batch_size):
  images.append(batch["observation"]["image_primary"][i, 0].numpy()) # [batch_size, window_size, rgb, width, height]

mediapy.show_video(images, fps=2)
# mediapy.show_video(images_wrist, fps=10)

########################################
constructing single val dataset: bridge_dataset
########################################


2024-07-08 20:11:22.400466: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-07-08 20:11:22.738188: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
W0000 00:00:1720462288.662468  645295 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "CropAndResize" attr { key: "T" value { type: DT_FLOAT } } attr { key: "extrapolation_value" value { f: 0 } } attr { key: "method" value { s: "bilinear" } } inputs { dtype: DT_FLOAT shape { dim { size: 1 } dim { size: 256 } dim { size: 256 } dim { size: -6 } } } inputs { dtype: DT_FLOAT shape { dim { size: -2 } dim { size: 4 } } } inputs { dtype: DT_INT32 shape { dim { size: -2 } } } inputs { dtype: DT_INT32 shape { dim { size: 2 } } } device { type: "CPU" vendor: "AuthenticAMD" model: "241" frequency: 2395 num_cores: 128 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4

ValueError: Image dimensions (1, 256, 256, 3) are invalid.

## Run Inference

Next, we will run inference over the images in the episode using the loaded model. 
Below we demonstrate setups for both goal-conditioned and language-conditioned training.
Note that we need to feed inputs of the correct temporal window size.

In [ ]:
print(episode['steps'].element_spec)
print(steps[0]['language_instruction'])
print(steps[0]['language_instruction_2'])
print(steps[0]['language_instruction_3'])
print(os.path.expanduser("~") + "/tensorflow_datasets")

In [ ]:
def rescale_to_range(tensor, max_values, min_values) -> torch.Tensor:
    # Scale the tensor to the new range [new_min, new_max]
    new_min = -torch.ones_like(tensor)
    new_max = torch.ones_like(tensor)
    rescaled_tensor = (tensor - new_min) / (new_max - new_min) * (max_values - min_values) + min_values
    return rescaled_tensor

# run inference loop, this model only uses 3rd person image observations for bridge
# collect predicted and true actions
pred_actions, true_actions = [], []
for step in tqdm.trange(len(images)):
    input_images = torch.from_numpy(np.moveaxis(images[step], 2, 0)).to(device).to(torch.float32).unsqueeze(0).unsqueeze(0)
    input_images_wrist = torch.from_numpy(np.moveaxis(images_wrist[step], 2, 0)).to(device).to(torch.float32).unsqueeze(0).unsqueeze(0)
    observation = {
        'image_primary': input_images,
        'image_wrist': input_images_wrist,
        'timestep_pad_mask': np.full((1, input_images.shape[1]), True, dtype=bool)
    }
    goal = steps[step]['language_instruction'].numpy().decode("utf-8")
    tokenized_goal = tokenizer(goal, return_tensors = 'pt', padding = "max_length", truncation = True, max_length = 77)
    tokenized_goal["input_ids"] = tokenized_goal["input_ids"].unsqueeze(0).to(device)
    tokenized_goal["attention_mask"] = tokenized_goal["attention_mask"].unsqueeze(0).to(device)
    task = {"language_instruction": tokenized_goal}
    
    # this returns *normalized* actions --> we need to unnormalize using the dataset statistics
    actions = agent({'observation': observation, 'task': task})
    train_stats = dataset_statistics["action"]
    # actions = rescale_to_range(actions[0], torch.Tensor(train_stats['p99']).to(device).to(torch.float32), torch.Tensor(train_stats['p01']).to(device).to(torch.float32)) # remove batch dim
    actions = rescale_to_range(actions[0], torch.Tensor(train_stats['max']).to(device).to(torch.float32), torch.Tensor(train_stats['min']).to(device).to(torch.float32)) # remove batch dim

    pred_actions.append(actions.cpu().numpy())
    final_window_step = step
    true_actions.append(steps[step]['action'])

## Visualize predictions and ground-truth actions

Finally, we will visualize the predicted actions in comparison to the groundtruth actions.

In [ ]:
import matplotlib.pyplot as plt

ACTION_DIM_LABELS = ['x', 'y', 'z', 'yaw', 'pitch', 'roll', 'grasp']

# build image strip to show above actions
img_strip = np.concatenate(np.array(images[::3]), axis=1)

# set up plt figure
figure_layout = [
    ['image'] * len(ACTION_DIM_LABELS),
    ACTION_DIM_LABELS
]
plt.rcParams.update({'font.size': 12})
fig, axs = plt.subplot_mosaic(figure_layout)
fig.set_size_inches([45, 10])

# plot actions
pred_actions = np.array(pred_actions).squeeze()
true_actions = np.array(true_actions).squeeze()
print(pred_actions.shape)
print(true_actions.shape)
for action_dim, action_label in enumerate(ACTION_DIM_LABELS):
  # actions have batch, horizon, dim, in this example we just take the first action for simplicity
  axs[action_label].plot(pred_actions[:, 0, action_dim], label='predicted action')
  axs[action_label].plot(true_actions[:, action_dim], label='ground truth')
  # axs[action_label].plot(true_actions[:, action_dim] - pred_actions[:, 0, action_dim], label='error')
  axs[action_label].set_title(action_label)
  axs[action_label].set_xlabel('Time in one episode')

axs['image'].imshow(img_strip)
axs['image'].set_xlabel('Time in one episode (subsampled)')
plt.legend()